# HyperStream Tutorial 2: Your own tools

HyperStream has a set of predefined tools in hyperstream.tools. However, it is possible to define your own tools and factors. In this tutorial, we show how to create a simple plugin that reads a CSV file. In this tutorial, we already created the tool and made all the configurations necessary for it to work. We will describe how this one was created, and how can you create a new one.

## Creating a plugin tool to read csv files

### 1. Create a folder in plugins

First of all, we need to create a new folder to contain our new tool. The new folder needs to be in the folder __plugins__, in this example __plugins/example/tools/csv_reader/__. We need to create an **\__init\__.py** file in every subfolder to be able to treat all the folders as a Python package.

    plugins/
        |- __init__.py
        |- one_plugin/
        |   |- __init__.py
        |   |- tools/
        |       |- __init__.py
        |       |- tool_a
        |           |- __init__.py
        |           |- 2017-06-20-v0.0.1.py
        |           |- 2017-06-22-v0.0.3.py
        |- another_plugin/
            |- __init__.py
            |- tools/
                |- tool_b/
                |   |- __init__.py
                |   |- 2017-06-20-v0.0.1.py
                |   |- 2017-06-22-v0.1.0.py
                |- tool_c/
                    |- __init__.py
                    |- 2017-06-20-v0.0.2.py

### 2. Write the plugin in Python

Then, we need to create a new Python file following the name convention &lt;year>-&lt;month\>-&lt;day\>-v&lt;version\>.&lt;subversion\>.&lt;subsubversion\>.py. In this example the file is called **2017-06-20_v0.0.1.py**

```Python
from hyperstream import Tool, StreamInstance
from hyperstream.utils import check_input_stream_count

from dateutil.parser import parse


class CsvReader(Tool):
    def __init__(self, filename):
        super(CsvReader, self).__init__(filename=filename)

    @check_input_stream_count(0)
    def _execute(self, sources, alignment_stream, interval):

        # Let's make the assumption that the first field is the timestamp

        first = True

        with open(self.filename, 'rU') as f:
            for line in f.readlines():
                if first:
                    first = False
                    continue
                elements = line.split(',')
                dt = parse(elements[0])
                if dt in interval:
                    yield StreamInstance(dt, map(float, elements[1:]))

```

### 3. Add HyperStream configuration

Now, it is necessary to add information about this plugin into the **hyperstream_config.json**. In particular, we need to add the following information in the plugin section:

- channel_id_prefix:
- channel_names: 
- path: path to the new plugin
- has_tools: If the new plugin has tools
- has_assets:

Next, we have an example of an configuration file with the new plugin:

```json
{
    "mongo": {
        "host": "localhost",
        "port": 27017,
        "tz_aware": true,
        "db": "hyperstream"
    },
    "plugins": [{
        "channel_id_prefix": "example",
        "channel_names": [],
        "path": "plugins/example",
        "has_tools": true,
        "has_assets": false
    }],
    "online_engine": {
        "interval": {
            "start": -60,
            "end": -10
        },
        "sleep": 5,
        "iterations": 100
    }
}

```

## Using the new tool

Now we can write some HyperStream code that uses the new plugin.

In [6]:
%load_ext watermark

import sys
sys.path.append("../") # Add parent dir in the Path

from hyperstream import HyperStream
from hyperstream import TimeInterval
from hyperstream.utils import UTC

from datetime import datetime

%watermark -v -m -p hyperstream -g

hs = HyperStream(loglevel=20)
print hs

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 2.7.6
IPython 5.3.0

hyperstream 0.3.0-beta

compiler   : GCC 4.8.4
system     : Linux
release    : 3.19.0-80-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit
Git hash   : 3dbe2e441f1e399f176bfce2fa0eed5cbba3a161
HyperStream version 0.3.0-beta, connected to mongodb://localhost:27017/hyperstream


## Loading the new plugin

After instantiating HyperStream, if the configuration of the plugin and the plugin are in the right place, we will be able to load our new tool **csv_reader**, specifying where is the input file

In [7]:
reader_tool = hs.plugins.example.tools.csv_reader('data/sea_ice.csv')

## Create a stream

Now we can create a stream to store in memory the results of applying the new tool

In [8]:
sea_ice_stream = hs.channel_manager.memory.get_or_create_stream("sea_ice")

## Query the stream

And finally we can query certain period of time to the tool and store the information in the created stream

In [9]:
ti = TimeInterval(datetime(1990, 1, 1).replace(tzinfo=UTC), datetime(1990, 11, 1).replace(tzinfo=UTC))

reader_tool.execute(sources=[], sink=sea_ice_stream, interval=ti)
for key, value in sea_ice_stream.window().items():
    print '[%s]: %s' % (key, value)

[1990-02-01 00:00:00+00:00]: [13.33, 2.15]
[1990-03-01 00:00:00+00:00]: [13.44, 2.71]
[1990-04-01 00:00:00+00:00]: [12.16, 5.1]
[1990-05-01 00:00:00+00:00]: [10.84, 7.37]
[1990-06-01 00:00:00+00:00]: [9.12, 10.26]
[1990-07-01 00:00:00+00:00]: [6.44, 12.17]
[1990-08-01 00:00:00+00:00]: [4.92, 13.95]
[1990-09-01 00:00:00+00:00]: [4.5, 14.3]
[1990-10-01 00:00:00+00:00]: [6.67, 13.71]
[1990-11-01 00:00:00+00:00]: [9.58, 11.24]
